# ĐỒ ÁN: Xử lý ngôn ngữ tự nhiên
Đinh Hoàng Linh Đan     - 19521309
Trần Nguyễn Quỳnh Anh   - 19521217  

In [1]:
from collections import defaultdict
import unicodedata as ud
import pandas as pd
import numpy as np
import math
import ast
import re

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

# Dataset

In [3]:
sentences = open('C:\\Users\Admin\\CS221.M11-POS_TAGGING_VIETNAMESE\\Dataset\\dataset.txt', encoding='utf-8').readlines()
tokenize_sentences = [sentence.split(' ') for sentence in sentences]

In [4]:
print('Số lượng câu trong Dataset:', len(sentences))
#sentences[0:len(sentences)]

Số lượng câu trong Dataset: 40


# Tách từ bằng Longest Matching

In [5]:
def syllablize(sentence):
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

In [6]:
def load_n_grams(path):
    with open(path, encoding='utf8') as f:
        words = f.read()
        words = ast.literal_eval(words)
    return words

In [7]:
def longest_matching(sentence, two, three):
    syllables = syllablize(sentence)
    syl_len = len(syllables)
    
    curr_id = 0
    word_list = []
    done = False
    
    while (curr_id < syl_len) and (not done):
        curr_word = syllables[curr_id]
        if curr_id >= syl_len - 1:
            word_list.append(curr_word)
            done = True
        else:
            next_word = syllables[curr_id + 1]
            pair_word = ' '.join([curr_word.lower(), next_word.lower()])
            if curr_id >= (syl_len - 2):
                if pair_word in two:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
            else:
                next_next_word = syllables[curr_id + 2]
                triple_word = ' '.join([pair_word, next_next_word.lower()])
                if triple_word in three:
                    word_list.append('_'.join([curr_word, next_word, next_next_word]))
                    curr_id += 3
                elif pair_word in two:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
    return word_list

In [9]:
two_word = load_n_grams('Resource\\two_word.txt')
three_word = load_n_grams('Resource\\three_word.txt')
#longest_matching('Hãy tách khỏi quá khứ và chôn vùi nó.', two_word, three_word)

In [10]:
with open('Tokens\\Longest_Matching_Tokens.txt', 'w', encoding='utf-8') as f:
    longest_matching_sentences = []
    for sentence in sentences:
        word_list = longest_matching(sentence, two_word, three_word)
        longest_matching_sentences.append(' '.join(word_list))
        for word in word_list: f.write(word + '\n')
        if sentence != sentences[-1]: f.write('\n')
    f.write('\n')
longest_matching_sentences[0:3]

['Tương_lai là hôm_nay .',
 'Hãy tách khỏi quá_khứ và chôn vùi nó .',
 'Nhưng ta hoàn_toàn không cần phải lo_lắng .']

In [11]:
count_longest_matching_compounds = 0
for sentence in longest_matching_sentences:
    for word in sentence.split():
        if '_' in word: count_longest_matching_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching:', count_longest_matching_compounds)

Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching: 156


# Tách từ bằng VnCoreNLP

In [6]:
%pip install vncorenlp

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [12]:
from vncorenlp import VnCoreNLP

In [13]:
client = VnCoreNLP(address='http://127.0.0.1', port=9001)

In [14]:
with open('Tokens\\vncore_tokens.txt', 'w', encoding='utf-8') as f:
    vncore_sentences = []
    for sentence in sentences:
        word_list = client.tokenize(sentence)[0]
        vncore_sentences.append(' '.join(word_list))
        for word in word_list: f.write(word + '\n')
        if sentence != sentences[-1]: f.write('\n')
    f.write('\n')
vncore_sentences[0:3]

['Tương_lai là hôm_nay .',
 'Hãy tách khỏi quá_khứ và chôn vùi nó .',
 'Nhưng ta hoàn_toàn không cần phải lo_lắng .']

In [15]:
count_vncore_compounds = 0
for sentence in vncore_sentences:
    for word in sentence.split():
        if '_' in word: count_vncore_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thư viện VnCoreNLP:', count_vncore_compounds)

Số lượng từ ghép khi tách từ bằng thư viện VnCoreNLP: 163


# Tách từ thủ công

In [16]:
with open('Tokens\manual_token.txt', 'r', encoding='utf-8') as f:
    manual_tokenize_sentences = []
    sentence = ''
    for word in f:
        if word == '\n': 
            manual_tokenize_sentences.append(sentence.strip())
            sentence = ''
        else: sentence += word.replace('\n', ' ')
manual_tokenize_sentences[0:3]

['Tương_lai là hôm_nay .',
 'Hãy tách khỏi quá_khứ và chôn vùi nó .',
 'Nhưng ta hoàn_toàn không cần phải lo_lắng .']

In [17]:
count_manual_tokenize_compounds = 0
for sentence in manual_tokenize_sentences:
    for word in sentence.split():
        if '_' in word: count_manual_tokenize_compounds += 1
print('Số lượng từ ghép khi tách từ thủ công:', count_manual_tokenize_compounds)

Số lượng từ ghép khi tách từ thủ công: 146


# Đánh giá kết quả tách từ

In [18]:
def count_correct_words(pred, source, n_grams=3):
    pred_words = pred.split()
    source_words = source.split()
    
    total_true, tp = 0, 0
    total_errors, fp = 0, 0
    
    idx = 0
    while idx < len(pred_words):
        if pred_words[idx] not in source_words[idx:(idx + n_grams)]: 
            if '_' in pred_words[idx]: fp += 1
            del pred_words[idx]
            total_errors += 1
        else: idx += 1
    
    idx = 0
    while idx < len(source_words):
        if source_words[idx] not in pred_words[idx:(idx + n_grams)]: 
            del source_words[idx]
        else: idx += 1
    
    if len(pred_words) < len(source_words): words = pred_words
    else: words = source_words
    
    for idx in range (len(words)):
        if pred_words[idx] == source_words[idx]:
            if '_' in pred_words[idx]: tp += 1 
            total_true += 1
                    
    return total_true, total_errors, tp, fp

In [19]:
def tokenize_evaluation(pred, source, n_grams=3):
    total_true = 0
    total_errors = 0
    total_words = 0
    
    pred_tp = 0
    pred_fp = 0
    
    for pred_sentence, source_sentence in zip(pred, source):
        total_words += len(source_sentence.split())
        if pred_sentence != source_sentence:
            true, error, tp, fp = count_correct_words(pred_sentence, source_sentence, n_grams)
            total_true += true 
            total_errors += error
            pred_tp += tp
            pred_fp += fp
        else:
            for word in source_sentence.split():
                if '_' in word: pred_tp += 1
                total_true += 1
    return {
        'Accuracy': total_true / total_words, 
        'Precision': pred_tp / (pred_tp + pred_fp),
        'Recall': pred_tp / count_manual_tokenize_compounds,
        'True Positive': pred_tp, 
        'False Positive': pred_fp,
        'Total True': total_true, 
        'Total Errors': total_errors
    }

In [20]:
longest_matching_evaluation = tokenize_evaluation(longest_matching_sentences, manual_tokenize_sentences)
vncore_evaluation = tokenize_evaluation(vncore_sentences, manual_tokenize_sentences)
pd.DataFrame(
    [longest_matching_evaluation, vncore_evaluation], 
    index = ['Longest Matching', 'VnCoreNLP']
).astype(object).T


,Longest Matching,VnCoreNLP
Accuracy,0.982487,0.929947
Precision,0.97973,0.915584
Recall,0.993151,0.965753
True Positive,145,141
False Positive,3,13
Total True,561,531
Total Errors,7,28


# Tạo ngữ liệu

In [21]:
manual_tokens = open('Tokens\\manual_token.txt', encoding='utf-8').readlines()
print('Số lượng từ:', len(manual_tokens))
manual_tokens[0:5]

Số lượng từ: 631


['Tương_lai\n', 'là\n', 'hôm_nay\n', '.\n', '\n']

In [22]:
longest_matching_tokens = open('Tokens\\Longest_Matching_Tokens.txt', encoding='utf-8').readlines()
print('Số lượng từ:', len(longest_matching_tokens))
longest_matching_tokens[0:5]

Số lượng từ: 629


['Tương_lai\n', 'là\n', 'hôm_nay\n', '.\n', '\n']

# Gán nhãn bằng VnCoreNLP

In [23]:
with open('Dataset\\label.txt', 'w', encoding='utf-8') as f:
    for word in manual_tokens:
        word = word.replace('\n', '')
        
        if '_' not in word: tag = client.pos_tag(word)
        else: tag = client.pos_tag(word.replace('_', ' '))
        
        if tag == []: f.write('\n')
        else: f.write(f'{word}\t{tag[0][0][1]}\n')
    f.write('\n')

# Chia dữ liệu thành tập Train và Test

In [24]:
label = open('Dataset\\label.txt', encoding='utf-8').readlines()
new_line_idx = [i for i, item in enumerate(manual_tokens) if item == '\n']

In [25]:
with open('Dataset\\label_train.txt', 'w', encoding='utf-8') as f:
    for line in label[:new_line_idx[32]]: f.write(line)
    f.write('\n')
    
with open('Dataset\\label_test.txt', 'w', encoding='utf-8') as f:
    for line in label[new_line_idx[32] + 1:]: f.write(line)

In [26]:
with open('Dataset\\train_words.txt', 'w', encoding='utf-8') as f:
    for line in label[:new_line_idx[32]]: f.write(re.sub('\t.*', '', line))
    f.write('\n')        
    
with open('Dataset\\test_words.txt', 'w', encoding='utf-8') as f:
    for line in label[new_line_idx[32] + 1:]: f.write(re.sub('\t.*', '', line))

# Preprocess Data

In [27]:
def preprocess(vocabs_dict, path):
    data = []
    file = open(path, encoding='utf-8').readlines()
    
    for index, word in enumerate(file):
        if not (word.lower()).split():
            word = '--n--'
            data.append(word)
            continue
        elif (word.lower()).strip() not in vocabs_dict:
            word = '--unk--'
            data.append(word)
            continue
        data.append(word.strip())
    return data

In [28]:
def plot_tag_counts(label):
    tags = [word_tag.split()[1] for word_tag in label if word_tag.split()]
    tag_counts = pd.DataFrame(tags)[0].value_counts()
    tag_counts.plot.bar(rot=0, width=0.7, legend=False, figsize=(15, 5))
    return pd.DataFrame(tag_counts).T.assign(Total=tag_counts.sum()) 

# Vocabs

In [29]:
vocabs = open('Resource\\vocabs.txt', encoding='utf-8').read().split('\n')
vocabs_dict = {}
index = 0

for word in sorted(vocabs): 
    if word not in vocabs_dict: 
        vocabs_dict[word] = index  
        index += 1

In [30]:
print('Số lượng từ vựng:', len(vocabs_dict.keys()))
count = 0

for key, value in vocabs_dict.items():
    print(f'{key}: {value}')
    count += 1
    if count > 20: break

Số lượng từ vựng: 54820
: 0
!: 1
": 2
#: 3
$: 4
%: 5
&: 6
': 7
'': 8
'40s: 9
'60s: 10
'70s: 11
'80s: 12
'86: 13
'90s: 14
'N: 15
'S: 16
'd: 17
'em: 18
'll: 19
'm: 20


# Tập Train

In [31]:
label_train = open('Dataset\\label_train.txt', encoding='utf-8').readlines()
print('Số lượng từ trong tập label_train:', len(label_train))
label_train[0:5]

Số lượng từ trong tập label_train: 520


['Tương_lai\tNp\n', 'là\tV\n', 'hôm_nay\tN\n', '.\tCH\n', '\n']

In [32]:
train_words = preprocess(vocabs_dict, 'Dataset\\train_words.txt')
print('Số lượng từ trong tập train_words:', len(train_words))
train_words[0:5]

Số lượng từ trong tập train_words: 520


['Tương_lai', 'là', 'hôm_nay', '.', '--n--']

In [33]:
print('Các từ không nằm trong vocabs', end=': ')
for word_tag, word in zip(label_train, train_words):
    if word == '--unk--': print(word_tag.split()[0], end=', ')
#plot_tag_counts(label_train)

Các từ không nằm trong vocabs: hủy, 1/3, khỏe, Tùy, thư_giãn, 

# Tập Test

In [34]:
label_test = open('Dataset\\label_test.txt', encoding='utf-8').readlines()
print('Số lượng từ trong tập test:', len(label_test))
label_test[0:5]

Số lượng từ trong tập test: 112


['Nếu\tC\n', 'chúng_ta\tP\n', 'đang\tR\n', 'mệt\tA\n', ',\tCH\n']

In [35]:
test_words = preprocess(vocabs_dict, 'Dataset\\test_words.txt')
print('Số lượng từ trong tập test_words:', len(test_words))
test_words[0:5]

Số lượng từ trong tập test_words: 112


['Nếu', 'chúng_ta', 'đang', 'mệt', ',']

In [36]:
print('Các từ không nằm trong vocabs', end=': ')
for word_tag, word in zip(label_test, test_words):
    if word == '--unk--': print(word_tag.split()[0], end=', ')
#plot_tag_counts(label_test)

Các từ không nằm trong vocabs: gây_ra, 

# Tags

In [37]:
def seperate_word_tag(word_tag, vocabs_dict): 
    if not word_tag.split():
        word = '--n--'
        tag = '--s--'
    else:
        word, tag = word_tag.split()
        if word not in vocabs_dict: word = '--unk--'
    return word, tag

In [38]:
def create_dictionaries(train_gold, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    prev_tag = '--s--' 
    for word_tag in train_gold:
        word, tag = seperate_word_tag(word_tag, vocab) 
        
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[(tag, word)] += 1
        tag_counts[tag] += 1
        prev_tag = tag
    return transition_counts, emission_counts, tag_counts


In [39]:
transition_counts, emission_counts, tag_counts = create_dictionaries(label_train, vocabs_dict)
states = sorted(tag_counts.keys())
print('Số nhãn:', len(states))
print(states)


Số nhãn: 16
['--s--', 'A', 'C', 'CH', 'Cc', 'E', 'L', 'M', 'N', 'Nc', 'Np', 'P', 'R', 'T', 'V', 'X']


In [42]:
print("Transition examples: ")
for example in list(transition_counts.items())[:10]:
    print(example)

Transition examples: 
(('--s--', 'Np'), 6)
(('Np', 'V'), 2)
(('V', 'N'), 20)
(('N', 'CH'), 11)
(('CH', '--s--'), 33)
(('--s--', 'R'), 2)
(('R', 'V'), 25)
(('V', 'V'), 51)
(('N', 'Cc'), 5)
(('Cc', 'V'), 5)


In [43]:
print("Emission examples: ")
for example in list(emission_counts.items())[:10]:
    print (example)


Emission examples: 
(('Np', '--unk--'), 6)
(('V', 'là'), 8)
(('N', 'hôm_nay'), 1)
(('CH', '.'), 33)
(('--s--', '--n--'), 33)
(('R', '--unk--'), 2)
(('V', 'tách'), 1)
(('V', 'khỏi'), 1)
(('N', 'quá_khứ'), 1)
(('Cc', 'và'), 11)


# Training

In [44]:
def seperate_word_tag(word_tag, vocabs_dict): 
    if not word_tag.split():
        word = '--n--'
        tag = '--s--'
    else:
        word, tag = word_tag.split()
        if word not in vocabs_dict: word = '--unk--'
    return word, tag

In [45]:
def create_dictionaries(train_gold, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    prev_tag = '--s--' 
    for word_tag in train_gold:
        word, tag = seperate_word_tag(word_tag, vocab) 
        
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[(tag, word)] += 1
        tag_counts[tag] += 1
        prev_tag = tag
    return transition_counts, emission_counts, tag_counts

In [46]:
transition_counts, emission_counts, tag_counts = create_dictionaries(label_train, vocabs_dict)
states = sorted(tag_counts.keys())
print('Số nhãn:', len(states))
print(states)

Số nhãn: 16
['--s--', 'A', 'C', 'CH', 'Cc', 'E', 'L', 'M', 'N', 'Nc', 'Np', 'P', 'R', 'T', 'V', 'X']


# Testing

In [47]:
def predict_pos(words, gold, emission_counts, vocabs_dict, states):
    num_correct = 0
    all_words = set(emission_counts.keys())
    
    for word, gold_tuple in zip(words, gold): 
        gold_tuple_list = gold_tuple.split()
        if len(gold_tuple_list) != 2: continue
        else: true_label = gold_tuple_list[1]
    
        count_final = 0
        pos_final = ''
        if word not in vocabs_dict: continue
        
        for pos in states:
            if (pos, word) not in emission_counts: continue
            count = emission_counts[(pos, word)]
            
            if count > count_final:
                count_final = count
                pos_final = pos
                    
        if pos_final == true_label: num_correct += 1
    accuracy = num_correct / len(gold)
    return accuracy

In [49]:
accuracy = predict_pos(train_words, label_train, emission_counts, vocabs_dict, states)
print('Độ chính xác trên tập train:', accuracy)

Độ chính xác trên tập train: 0.8884615384615384


In [48]:
accuracy = predict_pos(test_words, label_test, emission_counts, vocabs_dict, states)
print('Độ chính xác trên tập test:', accuracy)


Độ chính xác trên tập test: 0.5446428571428571


# Hidden Markov

In [50]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)
    
    A = np.zeros((num_tags, num_tags))
    trans_keys = set(transition_counts.keys())
    
    for i in range(num_tags):
        for j in range(num_tags):
            count = 0
            key = (all_tags[i], all_tags[j])
            if key in transition_counts: count = transition_counts[key]
                
            count_prev_tag = tag_counts[all_tags[i]]
            A[i, j] = (count + alpha) / (count_prev_tag + alpha * num_tags)
    return A


In [83]:
alpha = 0.001
for i in range(len(states)): tag_counts.pop(i, None)
    
A = create_transition_matrix(alpha, tag_counts, transition_counts)
df = pd.DataFrame(
    A[10:20, 10:20], 
    index = states[10:20], 
    columns = states[10:20]
)
df.head()

,Np,P,R,T,V,X
Np,0.000166,0.000166,0.000166,0.000166,0.332613,0.000166
P,0.000026,0.025656,0.128178,0.000026,0.358853,0.000026
R,0.000028,0.027793,0.111090,0.027793,0.694164,0.000028
T,0.000496,0.000496,0.000496,0.000496,0.992560,0.000496
V,0.000007,0.068965,0.027590,0.000007,0.351692,0.000007


In [84]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocabs):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(tag_counts)
    num_words = len(vocabs)
    
    B = np.zeros((num_tags, num_words))
    emis_keys = set(list(emission_counts.keys()))
    
    for i in range(num_tags):
        for j in range(num_words):
            count = 0
            key = (all_tags[i], vocabs[j])
            if key in emission_counts.keys(): count = emission_counts[key]
                
            count_tag = tag_counts[all_tags[i]]
            B[i, j] = (count + alpha) / (count_tag + alpha * num_words)
    return B


In [118]:
test_words_lower = []
for w in test_words:
    test_words_lower.append(w.lower())
test_words_lower

['nếu',
 'chúng_ta',
 'đang',
 'mệt',
 ',',
 'cơ_thể',
 'sẽ',
 'tự',
 'bắt',
 'ta',
 'phải',
 'ngủ',
 ',',
 'ngay',
 'khi',
 'ta',
 'đang',
 'bước_đi',
 '.',
 '--n--',
 'khi',
 'buồn_ngủ',
 'là',
 'tự_nhiên',
 'phải',
 'ngủ',
 ',',
 'dù',
 'ý_chí',
 'của',
 'chúng_ta',
 'có',
 'mạnh_mẽ',
 '.',
 '--n--',
 'hãy',
 'làm',
 'cho',
 'cơ_thể',
 'bạn',
 'mệt',
 'đến',
 'mức',
 'không_thể',
 'thức',
 'được',
 'nữa',
 '.',
 '--n--',
 'lo_lắng',
 'về',
 'chứng',
 'mất_ngủ',
 'thường',
 '--unk--',
 'những',
 'hậu_quả',
 'còn',
 'tệ_hại',
 'hơn',
 'bản_thân',
 'căn_bệnh',
 'này',
 '.',
 '--n--',
 'nhớ',
 'rằng',
 'chưa',
 'có',
 'ai',
 'chết',
 'vì',
 'thiếu',
 'ngủ',
 '.',
 '--n--',
 'chúng_ta',
 'có_thể',
 'nhịn',
 'ăn',
 ',',
 'nhịn',
 'uống',
 'lâu',
 'hơn',
 'là',
 'nhịn',
 'ngủ',
 '.',
 '--n--',
 'khi',
 'hoàn_toàn',
 'kiệt_sức',
 ',',
 'con_người',
 'vẫn',
 'có_thể',
 'ngủ',
 'được',
 'giữa',
 'trời',
 'sấm_sét',
 'đùng_đùng',
 'hay',
 'giữa',
 'bom',
 'đạn',
 'hiểm_nguy',
 'của',
 'chiến_t

In [119]:
cidx  = test_words_lower
rvals = states
cols = [vocabs_dict[word] for word in cidx]
rows = [states.index(tag) for tag in rvals]

for i in range(len(states)): tag_counts.pop(i, None)
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocabs_dict))

df = pd.DataFrame(B[np.ix_(rows, cols)], index=rvals, columns=cidx)
df.head()


,nếu,chúng_ta,đang,mệt,",",cơ_thể,sẽ,tự,bắt,ta,...,đùng_đùng,hay,giữa,bom,đạn,hiểm_nguy,của,chiến_tranh,.,--n--
--s--,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,...,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.000011,0.375780
A,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010
C,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,...,0.000015,0.014760,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015
CH,0.000010,0.000010,0.000010,0.000010,0.102239,0.000010,0.000010,0.000010,0.000010,0.000010,...,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.337365,0.000010
Cc,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,...,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015
